# Optimisation : GCC

## Calculs avec des constantes

Quand il n'y a que des constantes, on peut imaginer que gcc fait les calculs.

In [4]:
%%writefile progConstantes.c
#include <stdio.h>
int main () {
int i=3, j=5;
printf("4+5=%d\n",i+j);
return;}

Overwriting progConstantes.c


In [5]:
%%sh
arm-linux-gnueabi-gcc -S -static progConstantes.c
cat progConstantes.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progConstantes.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"4+5=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	mov	r3, #3
	str	r3, [fp, #-12]
	mov	r3, #5
	str	r3, [fp, #-8]
	ldr	r2, [fp, #-12]
	ldr	r3, [fp, #-8]
	add	r3, r2, r3
	ldr	r0, .L3
	mov	r1, r3
	bl	printf
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
.L4:
	.align	2
.L3:
	.word	.LC0
	.size	main, .-main
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


Et en demandant d'optimiser :

In [6]:
%%sh
arm-linux-gnueabi-gcc -O -S -static progConstantes.c
cat progConstantes.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 1
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progConstantes.c"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r3, lr}
	mov	r0, #1
	ldr	r1, .L2
	mov	r2, #8
	bl	__printf_chk
	ldmfd	sp!, {r3, pc}
.L3:
	.align	2
.L2:
	.word	.LC0
	.size	main, .-main
	.section	.rodata.str1.4,"aMS",%progbits,1
	.align	2
.LC0:
	.ascii	"4+5=%d\012\000"
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


## Sur les divisions

Est-ce que la division peut être optimisée ? pour les divisions par 2 ? pour les divisions par 10 ?

In [7]:
%%writefile progDivision.c
#include <stdio.h>
int main () {
int i;
scanf("%d",&i);
printf("%d / 10=%d\n",i,i/i);
return 0;}

Overwriting progDivision.c


In [10]:
%%sh
arm-linux-gnueabi-gcc -O2 -S -static progDivision.c
cat progDivision.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 2
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progDivision.c"
	.global	__aeabi_idiv
	.section	.text.startup,"ax",%progbits
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r4, lr}
	sub	sp, sp, #8
	add	r1, sp, #4
	ldr	r0, .L2
	bl	__isoc99_scanf
	ldr	r4, [sp, #4]
	mov	r1, r4
	mov	r0, r4
	bl	__aeabi_idiv
	mov	r2, r4
	mov	r3, r0
	ldr	r1, .L2+4
	mov	r0, #1
	bl	__printf_chk
	mov	r0, #0
	add	sp, sp, #8
	ldmfd	sp!, {r4, pc}
.L3:
	.align	2
.L2:
	.word	.LC0
	.word	.LC1
	.size	main, .-main
	.section	.rodata.str1.4,"aMS",%progbits,1
	.align	2
.LC0:
	.ascii	"%d\000"
	.space	1
.LC1:
	.ascii	"%d / 10=%d\012\000"
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


progDivision.c: Dans la fonction ‘main’:
progDivision.c:4:6: attention : ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]


In [11]:
%%writefile progDivision2.c
#include <stdio.h>
int main () {
int i;
scanf("%d",&i);
printf("%d / 10=%d\n",i,i/2);
return 0;}

Overwriting progDivision2.c


In [12]:
%%sh
arm-linux-gnueabi-gcc -S -static progDivision2.c
cat progDivision2.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progDivision2.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"%d\000"
	.align	2
.LC1:
	.ascii	"%d / 10=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	sub	r3, fp, #8
	ldr	r0, .L3
	mov	r1, r3
	bl	__isoc99_scanf
	ldr	r2, [fp, #-8]
	ldr	r3, [fp, #-8]
	mov	r1, r3, lsr #31
	add	r3, r1, r3
	mov	r3, r3, asr #1
	ldr	r0, .L3+4
	mov	r1, r2
	mov	r2, r3
	bl	printf
	mov	r3, #0
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
.L4:
	.align	2
.L3:
	.word	.LC0
	.word	.LC1
	.size	main, .-main
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


In [13]:
%%writefile progDivision10.c
#include <stdio.h>
int main () {
int i;
scanf("%d",&i);
printf("%d / 10=%d\n",i,i/10);
return 0;}

Overwriting progDivision10.c


In [14]:
%%sh
arm-linux-gnueabi-gcc -S -static progDivision10.c
cat progDivision10.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progDivision10.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"%d\000"
	.align	2
.LC1:
	.ascii	"%d / 10=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	sub	r3, fp, #8
	ldr	r0, .L3
	mov	r1, r3
	bl	__isoc99_scanf
	ldr	r2, [fp, #-8]
	ldr	r3, [fp, #-8]
	ldr	r1, .L3+4
	smull	r0, r1, r3, r1
	mov	r1, r1, asr #2
	mov	r3, r3, asr #31
	rsb	r3, r3, r1
	ldr	r0, .L3+8
	mov	r1, r2
	mov	r2, r3
	bl	printf
	mov	r3, #0
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
.L4:
	.align	2
.L3:
	.word	.LC0
	.word	1717986919
	.word	.LC1
	.size	main, .-main
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.

N.B. : cela marche parce qu'il y a des constantes ! (rappel, même i/i n'est pas optimisé)

## Les conditionnelles ?

In [15]:
%%writefile progConditionnelle.c
#include <stdio.h>
int main () {
int i=4;
if (i&1) {
  printf("4 est impair\n");}
else {
  printf("4 est pair\n");}
return 0;}

Overwriting progConditionnelle.c


In [16]:
%%sh
arm-linux-gnueabi-gcc -S -static progConditionnelle.c
cat progConditionnelle.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progConditionnelle.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"4 est impair\000"
	.align	2
.LC1:
	.ascii	"4 est pair\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	mov	r3, #4
	str	r3, [fp, #-8]
	ldr	r3, [fp, #-8]
	and	r3, r3, #1
	cmp	r3, #0
	beq	.L2
	ldr	r0, .L5
	bl	puts
	b	.L3
.L2:
	ldr	r0, .L5+4
	bl	puts
.L3:
	mov	r3, #0
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
.L6:
	.align	2
.L5:
	.word	.LC0
	.word	.LC1
	.size	main, .-main
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


version optimisée

In [17]:
%%sh
arm-linux-gnueabi-gcc -O -S -static progConditionnelle.c
cat progConditionnelle.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 1
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progConditionnelle.c"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r3, lr}
	ldr	r0, .L2
	bl	puts
	mov	r0, #0
	ldmfd	sp!, {r3, pc}
.L3:
	.align	2
.L2:
	.word	.LC0
	.size	main, .-main
	.section	.rodata.str1.4,"aMS",%progbits,1
	.align	2
.LC0:
	.ascii	"4 est pair\000"
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


## Les boucles ?

In [24]:
%%writefile progBoucle.c
#include <stdio.h>
int main () {
int i, s=0, a, r;
for(i=0;i<100;i++) {
  r=scanf("%d",&a);
  s=s+a;}
printf("Somme : %d\n",s);
return 0;}

Overwriting progBoucle.c


In [20]:
%%sh
arm-linux-gnueabi-gcc -S -static progBoucle.c
cat progBoucle.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progBoucle.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"%d\000"
	.align	2
.LC1:
	.ascii	"Somme : %d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 16
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #16
	mov	r3, #0
	str	r3, [fp, #-12]
	mov	r3, #0
	str	r3, [fp, #-16]
	b	.L2
.L3:
	sub	r3, fp, #20
	ldr	r0, .L5
	mov	r1, r3
	bl	__isoc99_scanf
	str	r0, [fp, #-8]
	ldr	r3, [fp, #-20]
	ldr	r2, [fp, #-12]
	add	r3, r2, r3
	str	r3, [fp, #-12]
	ldr	r3, [fp, #-16]
	add	r3, r3, #1
	str	r3, [fp, #-16]
.L2:
	ldr	r3, [fp, #-16]
	cmp	r3, #9
	ble	.L3
	ldr	r0, .L5+4
	ldr	r1, [fp, #-12]
	bl	printf
	mov	r3, #0
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
.L6:
	.ali

en demandant à gcc d'optimiser :

In [21]:
%%sh
arm-linux-gnueabi-gcc -O2 -S -static progBoucle.c
cat progBoucle.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 2
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progBoucle.c"
	.section	.text.startup,"ax",%progbits
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r4, r5, lr}
	mov	r4, #10
	sub	sp, sp, #12
	mov	r5, #0
.L2:
	ldr	r0, .L5
	add	r1, sp, #4
	bl	__isoc99_scanf
	ldr	r3, [sp, #4]
	subs	r4, r4, #1
	add	r5, r5, r3
	bne	.L2
	mov	r2, r5
	ldr	r1, .L5+4
	mov	r0, #1
	bl	__printf_chk
	mov	r0, r4
	add	sp, sp, #12
	ldmfd	sp!, {r4, r5, pc}
.L6:
	.align	2
.L5:
	.word	.LC0
	.word	.LC1
	.size	main, .-main
	.section	.rodata.str1.4,"aMS",%progbits,1
	.align	2
.LC0:
	.ascii	"%d\000"
	.space	1
.LC1:
	.ascii	"Somme : %d\012\000"
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


et avec clang ?

In [22]:
%%sh
clang -S progBoucle.c
cat progBoucle.s

	.file	"progBoucle.c"
	.text
	.globl	main
	.align	16, 0x90
	.type	main,@function
main:                                   # @main
	.cfi_startproc
# BB#0:
	pushq	%rbp
.Ltmp2:
	.cfi_def_cfa_offset 16
.Ltmp3:
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
.Ltmp4:
	.cfi_def_cfa_register %rbp
	subq	$32, %rsp
	movl	$0, -4(%rbp)
	movl	$0, -12(%rbp)
	movl	$0, -8(%rbp)
.LBB0_1:                                # =>This Inner Loop Header: Depth=1
	cmpl	$10, -8(%rbp)
	jge	.LBB0_4
# BB#2:                                 #   in Loop: Header=BB0_1 Depth=1
	leaq	.L.str, %rdi
	leaq	-16(%rbp), %rsi
	movb	$0, %al
	callq	__isoc99_scanf
	movl	%eax, -20(%rbp)
	movl	-12(%rbp), %eax
	addl	-16(%rbp), %eax
	movl	%eax, -12(%rbp)
# BB#3:                                 #   in Loop: Header=BB0_1 Depth=1
	movl	-8(%rbp), %eax
	addl	$1, %eax
	movl	%eax, -8(%rbp)
	jmp	.LBB0_1
.LBB0_4:
	leaq	.L.str1, %rdi
	movl	-12(%rbp), %esi
	movb	$0, %al
	callq	printf
	movl	$0, %esi
	movl	%eax, -24(%rbp)         # 4-byte Spill
	movl	%esi, %e

In [25]:
%%sh
clang -O -S progBoucle.c
cat progBoucle.s

	.file	"progBoucle.c"
	.text
	.globl	main
	.align	16, 0x90
	.type	main,@function
main:                                   # @main
	.cfi_startproc
# BB#0:
	pushq	%rbp
.Ltmp4:
	.cfi_def_cfa_offset 16
	pushq	%r14
.Ltmp5:
	.cfi_def_cfa_offset 24
	pushq	%rbx
.Ltmp6:
	.cfi_def_cfa_offset 32
	subq	$16, %rsp
.Ltmp7:
	.cfi_def_cfa_offset 48
.Ltmp8:
	.cfi_offset %rbx, -32
.Ltmp9:
	.cfi_offset %r14, -24
.Ltmp10:
	.cfi_offset %rbp, -16
	movl	$100, %ebp
	leaq	12(%rsp), %r14
	xorl	%ebx, %ebx
	.align	16, 0x90
.LBB0_1:                                # =>This Inner Loop Header: Depth=1
	movl	$.L.str, %edi
	xorl	%eax, %eax
	movq	%r14, %rsi
	callq	__isoc99_scanf
	addl	12(%rsp), %ebx
	decl	%ebp
	jne	.LBB0_1
# BB#2:
	movl	$.L.str1, %edi
	xorl	%eax, %eax
	movl	%ebx, %esi
	callq	printf
	xorl	%eax, %eax
	addq	$16, %rsp
	popq	%rbx
	popq	%r14
	popq	%rbp
	ret
.Ltmp11:
	.size	main, .Ltmp11-main
	.cfi_endproc

	.type	.L.str,@object          # @.str
	.section	.rodata.str1.1,"aMS",@progbits,1
.L.str:
	.asciz	"%d"
	.s

## Les fonctions ?

In [26]:
%%writefile progFonction.c
#include <stdio.h>

int somme(int a,int b) {
return a+b;}

int main () {
int i, j;
scanf("%d",&i);
scanf("%d",&j);
printf("4+5=%d\n",somme(i,j));
return;}

Overwriting progFonction.c


In [27]:
%%sh
arm-linux-gnueabi-gcc -S -static progFonction.c
cat progFonction.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progFonction.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	@ link register save eliminated.
	str	fp, [sp, #-4]!
	add	fp, sp, #0
	sub	sp, sp, #12
	str	r0, [fp, #-8]
	str	r1, [fp, #-12]
	ldr	r2, [fp, #-8]
	ldr	r3, [fp, #-12]
	add	r3, r2, r3
	mov	r0, r3
	add	sp, fp, #0
	ldmfd	sp!, {fp}
	bx	lr
	.size	somme, .-somme
	.section	.rodata
	.align	2
.LC0:
	.ascii	"%d\000"
	.align	2
.LC1:
	.ascii	"4+5=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	sub	r3, fp, #12
	ldr	r0, .L5
	mov	r1, r3
	bl	__isoc99_s

en demandant d'optimiser

In [28]:
%%sh
arm-linux-gnueabi-gcc -O2 -S -static progFonction.c
cat progFonction.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 2
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"progFonction.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	@ link register save eliminated.
	add	r0, r0, r1
	bx	lr
	.size	somme, .-somme
	.section	.text.startup,"ax",%progbits
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r4, lr}
	ldr	r4, .L3
	sub	sp, sp, #8
	mov	r1, sp
	mov	r0, r4
	bl	__isoc99_scanf
	add	r1, sp, #4
	mov	r0, r4
	bl	__isoc99_scanf
	ldmia	sp, {r2, r3}
	ldr	r1, .L3+4
	add	r2, r2, r3
	mov	r0, #1
	bl	__printf_chk
	add	sp, sp, #8
	ldmfd	sp!, {r4, pc}
.L4:
	.align	2
.L3:
	.word	.LC0
	.word	.LC1
	.size	main, .-main
	.section	.rodata.str1.4

progFonction.c: Dans la fonction ‘main’:
progFonction.c:8:6: attention : ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]
progFonction.c:9:6: attention : ignoring return value of ‘scanf’, declared with attribute warn_unused_result [-Wunused-result]
